# XGBoost
XGBoost is short for “Extreme Gradient Boosting”, where the term “Gradient Boosting” is proposed in the paper Greedy Function Approximation: A Gradient Boosting Machine, by Friedman. XGBoost is based on this original model. This is a tutorial on gradient boosted trees, and most of the content is based on these slides by the author of xgboost.

### Loss and Regularization
Based on different understandings of prediction $y_{i}$ we can have different problems, such as regression, classification, ordering, etc. We need to find a way to find the best parameters given the training data. In order to do so, we need to define a so-called `objective function`, to measure the performance of the model given a certain set of parameters.

A very important fact about objective functions is they `must always` contain two parts: training loss and regularization.
$$obj(\Theta ) = L(\Theta) + \Omega (\Theta)$$

#### 1. Loss function (L) <br/>
The training loss measures how predictive our model is on training data. <br/>
- mean square error(MSE)
$$L(\Theta) = \sum_{i}^{ }(y_{i} - \widehat{y_{i}})^{2}$$

- logistic loss for logistic regression
$$L(\Theta) =\sum_{i}^{ } [y_{i} ln (1+e^{-\widehat{y_{i}}} ) + (1-y_{i} )ln(1+e^{\widehat{y_{i}}})]$$

#### 2. Regularization term ($\Omega$)
It controls the complexity of the model, which helps us to avoid overfitting.

### Tree Ensemble
The tree ensemble model is a set of classification and regression trees (CART). We classify the elements into different leaves, and assign them the score on the corresponding leaf. A CART is a bit different from decision trees, where the leaf only contains decision values. In CART, a real score is associated with each of the leaves, which gives us richer interpretations that go beyond classification.
<br/>
<img src='./resources/cart.png' alt='boosting' style="width: 700px;"/>
<br/>
Usually, a single tree is not strong enough to be used in practice. What is actually used is the so-called tree ensemble model, which sums the prediction of multiple trees together.
<br/>
<img src='./resources/twocarts.png' alt='boosting' style="width: 700px;"/>
<br/><br/>

The prediction scores of each individual tree are summed up to get the final score. If you look at the example, an important fact is that the two trees try to complement each other. Mathematically, we can write our model in the form
$$\widehat{y_{i}} = \sum_{k=1}^{K}f_{k}(x_{i}), fk\in F $$
<br/>
where K is the number of trees, f is a function in the functional space F, and F is the set of all possible CARTs. Therefore our objective to optimize can be written as
$$obj(\Theta ) = \sum_{i=1}^{n}(y_{i} , \widehat{y_{i}}) + \sum_{i=1}^{t}\Omega (f_{i})$$

### Tree Boosting
 How should we learn the trees? The answer is, as is always for all supervised learning models: Define an objective function, and optimize it!

Assume we have the following objective function :
$$obj  = \sum_{i=1}^{n}L(y_{i} , \widehat{y_{i}}^{(t)}) + \sum_{i=1}^{t}\Omega (f_{i})$$


### Training
##### 1. What are the parameters of trees? <br/>
Functions $f_{i}$, with each containing the structure of the tree and the leaf scores. <br/><br/>
This is much harder than traditional optimization problem where you can take the gradient and go. It is not easy to train all the trees at once. Instead, we use an additive strategy: fix what we have learned, and add one new tree at a time. 

We write the prediction value at step `t` as $\widehat{y_{i}}^{(t)}$ :


$$
{\displaystyle {\begin{aligned}
\widehat{y_{i}}^{(0)} &= 0 \\
\widehat{y_{i}}^{(1)} &= f_{1}(x_{i}) = \widehat{y_{i}}^{(0)} + f_{1}(x_{i})\\
\widehat{y_{i}}^{(2)} &= f_{1}(x_{i}) + f_{2}(x_{i})= \widehat{y_{i}}^{(1)} + f_{2}(x_{i})\\
&... \\
\widehat{y_{i}}^{(t)} &= \sum_{k=1}^{t}f_{k}(x_{i}) = \widehat{y_{i}}^{(t-1)} + f_{t}(x_{i}) 
\end{aligned}}
}$$




###### 2. Which tree do we want at each step? A natural thing is to add the one that optimizes our objective.
$$
{\displaystyle {\begin{aligned}
obj^{(t)} &= \sum_{i=1}^{n} l (y_{i},\widehat{y}_{i}^{(t)}) + \sum_{i=1}^{t}\Omega (f_{i}) \\
&= \sum_{i=1}^{n} l (y_{i},\widehat{y}_{i}^{(t-1)} + f_{t}(x_{i})) + \Omega (f_{i}) + \mathit{constant} \\
&= \sum_{i=1}^{n} [ l (y_{i}, \widehat{y}_{i}^{(t-1)}) + g_{i}f_{t}(x_{i}) + \frac{1}{2}h_{i}f_{t}^{2}(x_{i})] + \Omega (f_{t}) + \mathit{constant}
\end{aligned}}
}$$

where, <br/>
$$g_{i} = \partial _{\widehat{y}_{i}^{(t-1)}} l( y_{i}, \widehat{y}_{i}^{(t-1)})$$
$$h_{i} = \partial _{\widehat{y}_{i}^{(t-1)}}^{2} l( y_{i}, \widehat{y}_{i}^{(t-1)})$$ <br/>

This becomes our optimization goal for the new tree. One important advantage of this definition is that it only depends on $g_{i}$ and $h_{i}$. This is how xgboost can support custom loss functions. We can optimize every loss function, including logistic regression and weighted logistic regression, using exactly the same solver that takes $g_{i}$ and $h_{i}$ as input!

### Model Complexity (regularization)

We need to define the complexity of the tree $\Omega (f_{t})$. In order to do so, let us first refine the definition of the tree $f(x)$ as:
$$f_{t}(x) = w_{q(x)}, w\in R^{T}, q:R^{d} \rightarrow \left \{  1,2, ..., T\right \}.$$
where,<br/>
$w$ is the vector of scores on leaves,<br/>
$q$ is a function assigning each data point to the corresponding leaf,<br/>
$T$ is the number of leaves. <br/><br/>

in XGBoost, we define the complexity as
$$\Omega (f) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T}w_{j}^{2}$$



### The Structure Score
After reformalizing the tree model, we can write the objective value with the t-th tree as:
$$
{\displaystyle {\begin{aligned}
obj^{(t)} &\approx \sum_{i=1}^{n}[g_{i}w_{q(x_{i})} +  \frac{1}{2}h_{i}w_{q(x_{i})}^{2}] + \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T}w_{j}^{2} \\
& = \sum_{j=1}^{T}[(\sum_{i \in I_{j}}^{ } g_{i})w_{j} + \frac{1}{2}(\sum_{i \in I_{j}}^{ }h_{i} + \lambda )w_{j}^{2}] + \gamma T \\
& = \sum_{j=1}^{T}[G_{i}w_{j} + \frac{1}{2}(H_{i}+ \lambda )w_{j}^{2}] + \gamma T 
\end{aligned}}
}$$
<br/>
where,<br/>
$I_{j} = \left \{ i|q(x_{i} = j) \right \}$ is the set of indices of data points assigned to the j-th leaf. <br/>
$G_{j} = \sum_{i \in I_{j}}^{}g_{i}$  <br/>
$H_{j} = \sum_{i \in I_{j}}^{}h_{i}$  <br/>

Notice that in the second line we have changed the index of the summation because all the data points on the same leaf get the same score. In this equation $w_{j}$  are independent with respect to each other, the form $G_{i}w_{j} + \frac{1}{2}(H_{i}+ \lambda )w_{j}^{2}$ is quadratic and the best $w_{j}$ for a given structure $q(x)$ and the best objective reduction we can get is:
$$w_{j}^{*} = - \frac{G_{j}}{H_{j}+\lambda }$$<br/>
$$obj^{*} = -\frac{1}{2}\sum_{j=1}^{T}\frac{G_{j}^{2}}{H_{j} + \lambda} + \gamma T$$

It measures `how good` a tree structure $q(x)$ is.

### Learn the tree structure
Now that we have a way to measure how good a tree is, ideally we would enumerate all possible trees and pick the best one. In practice this is intractable, so we will try to optimize one level of the tree at a time. Specifically we try to split a leaf into two leaves, and the score it gains is
$$\mathit{Gain} = \frac{1}{2}\left [ \frac{G_{L}^{2}}{H_{L}+\lambda } + \frac{G_{R}^{2}}{H_{R}+\lambda }   - \frac{(G_{L} + G_{R})^{2}}{H_{L} +H_{R} + \lambda  } \right] - \gamma $$

This formula can be decomposed as 
* the score on the new left leaf 
* the score on the new right leaf 
* The score on the original leaf 
* regularization on the additional leaf. 

We can see an important fact here: if the gain is smaller than γ we would do better not to add that branch. This is exactly the `pruning` techniques in tree based models! 